# UKB RAP Dataset Filtering

This notebook outlines the filtering of UKB for individuals IDs based on filtering criteria.
It provides a more detailed, fine-scale filtering system than the one implemented in the Cohort Explorer.

It filters for individuals with disease status using
1. Hospital diagnoses (ICD10/9)
2. Death cause record mention of disease X
3. Self-reported mention of disease X
4. First occurrence of disease X (if suitable)

It can also filter for individuals who have a certain operation record as well.

Code as per: https://github.com/dnanexus/OpenBio/blob/master/UKB_notebooks/ukb-rap-pheno-basic.ipynb

In [1]:
# Import packages
import pyspark
import dxpy
import dxdata

In [2]:
dxdata.__version__

'0.41.0'

In [3]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [4]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

## Access the Dataset File

In [5]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [6]:
#Print out the entities within that dataset (i.e subtables including main entity = 'participant')
dataset.entities

[<Entity "participant">,
 <Entity "covid19_result_england">,
 <Entity "covid19_result_scotland">,
 <Entity "covid19_result_wales">,
 <Entity "gp_clinical">,
 <Entity "gp_scripts">,
 <Entity "gp_registrations">,
 <Entity "hesin">,
 <Entity "hesin_diag">,
 <Entity "hesin_oper">,
 <Entity "hesin_critical">,
 <Entity "hesin_maternity">,
 <Entity "hesin_delivery">,
 <Entity "hesin_psych">,
 <Entity "death">,
 <Entity "death_cause">,
 <Entity "omop_death">,
 <Entity "omop_device_exposure">,
 <Entity "omop_note">,
 <Entity "omop_observation">,
 <Entity "omop_drug_exposure">,
 <Entity "omop_observation_period">,
 <Entity "omop_person">,
 <Entity "omop_procedure_occurrence">,
 <Entity "omop_specimen">,
 <Entity "omop_visit_detail">,
 <Entity "omop_visit_occurrence">,
 <Entity "omop_dose_era">,
 <Entity "omop_drug_era">,
 <Entity "omop_condition_era">,
 <Entity "omop_condition_occurrence">,
 <Entity "omop_measurement">,
 <Entity "olink_instance_0">,
 <Entity "olink_instance_2">,
 <Entity "olink_

In [7]:
participant = dataset['participant']

In [8]:
# Returns all field objects for a given UKB showcase field id

def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given UKB showcase field id

def field_names_for_id(field_id):
    return [f.name for f in fields_for_id(field_id)]

In [22]:
#This provides all the field-names for all the instances/arrays of a single field-name.

#Hospital ICD10 codes:
field_names_for_id(41270)

#Hospital ICD9 codes:
field_names_for_id(41271)

#Death register 
field_names_for_id(40001)
field_names_for_id(40002)

#Verbal interview non-cancer illness codes
non_cancer_illness_fields = sum([field_names_for_id(20002)],[]) 

#First occurrence of HF (I50) and T2D (E11)
first_hf_occurrence= field_names_for_id(131354)
first_t2d_occurrence= field_names_for_id(130708)

#Verbal interview self-reported operation codes
operations_selfreport = sum([field_names_for_id(20004)],[]) 

#Verbal interview self-reported operation date codes
operations_selfreport_dates = sum([field_names_for_id(20010)],[]) 

/tmp/ipykernel_82/339987039.py:7: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return sorted(fields, key=lambda f: LooseVersion(f.name))


In [23]:
#This outlines the field names of the participants entity that I will be using..
participant_field_names_of_interest = ['eid', 'p41270','p41271'] + non_cancer_illness_fields + operations_selfreport + operations_selfreport_dates + first_hf_occurrence + first_t2d_occurrence

## Extraction of Chosen Fields from Participant Entity -> Spark DataFrame

In [24]:
main_df = participant.retrieve_fields(names=participant_field_names_of_interest, engine=dxdata.connect(), coding_values='replace')

In [12]:
#Show the head
#main_df.show(5, truncate=False)

## Filtering for Participant IDs in the Main Participant Entity

In [38]:
from pyspark.sql import functions as F

#Change your ICD-10 code for filtering here

#For HF
# main_icd10 = main_df.filter(
#     (F.array_contains(main_df.p41270,'I50.0 Congestive heart failure'))|
#     (F.array_contains(main_df.p41270,'I50.1 Left ventricular failure'))|
#     (F.array_contains(main_df.p41270,'I50.9 Heart failure, unspecified')) 
# )

#For hypertensive heart disease
# main_icd10 = main_df.filter(
#     (F.array_contains(main_df.p41270,'I11.0 Hypertensive heart disease with (congestive) heart failure'))|
#     (F.array_contains(main_df.p41270,'I11.9 Hypertensive heart disease without (congestive) heart failure'))
# )

#For HCM
# main_icd10 = main_df.filter(
#     (F.array_contains(main_df.p41270,'I42.1 Obstructive hypertrophic cardiomyopathy'))|
#     (F.array_contains(main_df.p41270,'I42.2 Other hypertrophic cardiomyopathy'))
# )

#For T2D
main_icd10 = main_df.filter(
    (F.array_contains(main_df.p41270,'E11.0 With coma'))|
    (F.array_contains(main_df.p41270,'E11.1 With ketoacidosis'))|
    (F.array_contains(main_df.p41270,'E11.2 With renal complications'))|
    (F.array_contains(main_df.p41270,'E11.3 With ophthalmic complications'))|
    (F.array_contains(main_df.p41270,'E11.4 With neurological complications'))|
    (F.array_contains(main_df.p41270,'E11.5 With peripheral circulatory complications'))|
    (F.array_contains(main_df.p41270,'E11.6 With other specified complications'))|
    (F.array_contains(main_df.p41270,'E11.7 With multiple complications'))|
    (F.array_contains(main_df.p41270,'E11.8 With unspecified complications'))|
    (F.array_contains(main_df.p41270,'E11.9 Without complications'))
)

main_icd10.count()

44437

In [39]:
#Filter within ICD-9 summary diagnoses

#For HF
# main_icd9 = main_df.filter(
#     (F.array_contains(main_df.p41271,'4280 Congestive heart failure'))|
#     (F.array_contains(main_df.p41271,'4281 Left heart failure'))|
#     (F.array_contains(main_df.p41271,'4289 Heart failure, unspecified'))
# )

#For HCM
# main_icd9 = main_df.filter(
#     (F.array_contains(main_df.p41271,'4251 Hypertrophic obstructive cardiomyopathy'))
# )

#For T2D
main_icd9 = main_df.filter(
    (F.array_contains(main_df.p41271,'25000 Diabetes mellitus without mention of complication (adult-onset type)'))|
    (F.array_contains(main_df.p41271,'25010 Diabetes with ketoacidosis (adult-onset type)'))
)

main_icd9.count()

41

In [40]:
#Filter within Interview/Self-Reported

from functools import reduce
#Also filter for individuals with  via verbal interview
# Step 1: Identify columns starting with 'p2002'
p2002_cols = [col for col in main_df.columns if col.startswith('p20002')]

# Step 2: Create condition for each column and change here too if you want to filter for other diseases

#For HF
# conditions = [F.col(col).contains('heart failure') for col in p2002_cols]

#For HCM
# conditions = [F.col(col).contains('hypertrophic cardiomyopathy (hcm / hocm)') for col in p2002_cols]

#For T2D
conditions = [F.col(col).contains('type 2 diabetes') for col in p2002_cols]


# Step 3: Combine all conditions
all_conditions = reduce(lambda a, b: a | b, conditions)

# Step 4: Apply the filter
interview = main_df.filter(all_conditions)
interview.count()


5164

# Filtering for Participant IDs using First Occurrence Data Fields

In [41]:
#For HF
# main_firstocc = main_df.filter(
#     (~ F.isnull(main_df.p131354))
# )

#For T2D
main_firstocc = main_df.filter(
    (~ F.isnull(main_df.p130708))
)

main_firstocc.count()

47076

## Filter for Participant IDs via Death Cause in Registry - ARCHIVED

In [ ]:
#Filter within primary and secondary causes of death via ICD-10 codes.
#This is absent from the participant table so using raw death cause record data instead.

#Change your ICD-10 code for filtering here

#For HF
# death1_icd10 = main_df.filter(
#     (F.array_contains(main_df.p40001,'I50.0 Congestive heart failure'))|
#     (F.array_contains(main_df.p40001,'I50.1 Left ventricular failure'))|
#     (F.array_contains(main_df.p40001,'I50.9 Heart failure, unspecified')) 
# )

#For hypertensive heart disease
# death1_icd10 = main_df.filter(
#     (F.array_contains(main_df.p40001,'I11.0 Hypertensive heart disease with (congestive) heart failure'))|
#     (F.array_contains(main_df.p40001,'I11.9 Hypertensive heart disease without (congestive) heart failure'))
# )

#For HCM
# death1_icd10 = main_df.filter(
#     (F.array_contains(main_df.p40001,'I42.1 Obstructive hypertrophic cardiomyopathy'))|
#     (F.array_contains(main_df.p40001,'I42.2 Other hypertrophic cardiomyopathy'))
# )

# death1_icd10.count()

In [ ]:
#For HF
# death2_icd10 = main_df.filter(
#     (F.array_contains(main_df.p40002,'I50.0 Congestive heart failure'))|
#     (F.array_contains(main_df.p40002,'I50.1 Left ventricular failure'))|
#     (F.array_contains(main_df.p40002,'I50.9 Heart failure, unspecified')) 
# )

#For hypertensive heart disease
# death2_icd10 = main_df.filter(
#     (F.array_contains(main_df.p40002,'I11.0 Hypertensive heart disease with (congestive) heart failure'))|
#     (F.array_contains(main_df.p40002,'I11.9 Hypertensive heart disease without (congestive) heart failure'))
# )

#For HCM
# death2_icd10 = main_df.filter(
#     (F.array_contains(main_df.p40002,'I42.1 Obstructive hypertrophic cardiomyopathy'))|
#     (F.array_contains(main_df.p40002,'I42.2 Other hypertrophic cardiomyopathy'))
# )

# death2_icd10.count()

## Filtering for Participant IDs in the Disease Cause Record Entity

In [42]:
#I also extract the death cause record data for filtering
death_cause_record = dataset['death_cause']

In [43]:
# Returns all field objects for a given title keyword

def fields_by_title_keyword(keyword):
    from distutils.version import LooseVersion
    fields = list(death_cause_record.find_fields(lambda f: keyword.lower() in f.title.lower()))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given title keyword

def field_names_by_title_keyword(keyword):
    return [f.name for f in fields_by_title_keyword(keyword)]

# Returns all field titles for a given title keyword

def field_titles_by_title_keyword(keyword):
    return [f.title for f in fields_by_title_keyword(keyword)]

In [44]:
death_df = death_cause_record.retrieve_fields(names=['eid','cause_icd10'], engine=dxdata.connect(), coding_values='replace')

In [45]:
#For HF
# death_df_icd10 = death_df.filter(
#     (death_df.cause_icd10.contains('I50.0 Congestive heart failure'))|
#     (death_df.cause_icd10.contains('I50.1 Left ventricular failure'))|
#     (death_df.cause_icd10.contains('I50.9 Heart failure, unspecified'))
# )

#For hypertensive heart disease
# death_df_icd10 = death_df.filter(
#     (death_df.cause_icd10.contains('I11.0 Hypertensive heart disease with (congestive) heart failure'))|
#     (death_df.cause_icd10.contains('I11.9 Hypertensive heart disease without (congestive) heart failure'))
# )

#For HCM
# death_df_icd10 = death_df.filter(
#     (death_df.cause_icd10.contains('I42.1 Obstructive hypertrophic cardiomyopathy'))|
#     (death_df.cause_icd10.contains('I42.2 Other hypertrophic cardiomyopathy'))
# )

#For T2D
death_df_icd10 = death_df.filter(
    (death_df.cause_icd10.contains('E11.0 With coma'))|
    (death_df.cause_icd10.contains('E11.1 With ketoacidosis'))|
    (death_df.cause_icd10.contains('E11.2 With renal complications'))|
    (death_df.cause_icd10.contains('E11.3 With ophthalmic complications'))|
    (death_df.cause_icd10.contains('E11.4 With neurological complications'))|
    (death_df.cause_icd10.contains('E11.5 With peripheral circulatory complications'))|
    (death_df.cause_icd10.contains('E11.6 With other specified complications'))|
    (death_df.cause_icd10.contains('E11.7 With multiple complications'))|
    (death_df.cause_icd10.contains('E11.8 With unspecified complications'))|
    (death_df.cause_icd10.contains('E11.9 Without complications'))
)

death_df_icd10.count()

2382

## Miscellaneous Sources e.g CMR

In [27]:
#For HCM
# misc_names_of_interest = ['eid', 'p24140']
# misc_df = participant.retrieve_fields(names=misc_names_of_interest, engine=dxdata.connect(), coding_values='replace')

# #For HCM
# misc_pass = misc_df.filter(
#     (misc_df.p24140 >= 15)
# )

# misc_pass.count()

ValueError: Field name(s) not found: ['p24140']

## Aggregation of Different Data Sources

In [46]:
%%capture
## Filter for all overlapping EID in the three filtered Spark dataframe
## Convert to Pandas DataFrame and then overlap


main_icd10_pd = main_icd10.toPandas()
main_icd9_pd = main_icd9.toPandas()
firstocc_pd= main_firstocc.toPandas()
death_df_icd10_pd = death_df_icd10.toPandas()
interview_pd = interview.toPandas()

In [47]:
import pandas as pd
overlap_eid = pd.concat([death_df_icd10_pd['eid'],
                        main_icd10_pd['eid'],
                         main_icd9_pd['eid'],
                         interview_pd['eid'], #Include if using verbal interview as filter
                         firstocc_pd['eid'] #Include if using first occurrence as filter
                        ]).unique()

In [48]:
overlap_eid

array(['3999490', '5935029', '2986828', ..., '5946661', '5988229',
       '6025513'], dtype=object)

In [49]:
import numpy as np
# pd.DataFrame(overlap_eid).to_csv('ukb_rap_HF_20240209.csv', index=False)
#pd.DataFrame(overlap_eid).to_csv('ukb_rap_HCM_20240209.csv', index=False
pd.DataFrame(overlap_eid).to_csv('ukb_rap_T2D_20240209.csv', index=False)

# Filter for Individuals with Self-Reported Operations

In [ ]:
#Filter within Interview/Self-Reported for Operations
from pyspark.sql import functions as F

from functools import reduce
#Also filter for individuals with  via verbal interview
# Step 1: Identify columns starting with 'p2002'
p2004_cols = [col for col in main_df.columns if col.startswith('p20004')]

# Step 2: Create condition for each column and change here too if you want to filter for other diseases

#For ICD
#conditions = [F.col(col).contains('defibrillator/icd insertion') for col in p2004_cols]

#For Heart Transplant
conditions = [F.col(col).contains('heart transplant') for col in p2004_cols]

# Step 3: Combine all conditions
all_conditions = reduce(lambda a, b: a | b, conditions)

# Step 4: Apply the filter
interview_operations = main_df.filter(all_conditions)
interview_operations.count()

In [ ]:
#Export the IDs of those who have either self-reported ICD implantation or Heart Transplant
import pandas as pd
interview_operations_pd = interview_operations.toPandas()
# pd.DataFrame(interview_operations_pd['eid'].unique()).to_csv('ukb_rap_SelfReportICD_20240209.csv', index=False)
pd.DataFrame(interview_operations_pd['eid'].unique()).to_csv('ukb_rap_SelfReportHeartTransplant_20240209.csv', index=False)

In [35]:
#Extract the fields corresponding to the operation code and the interpolated year of operation from self-report but only for these IDs

opids_ht = pd.read_csv('ukb_rap_SelfReportHeartTransplant_20240209.csv')
opids_icd = pd.read_csv('ukb_rap_SelfReportICD_20240209.csv')

opids=pd.concat([opids_ht, opids_icd], axis=0)
opids.rename(columns={"0": "eid"}, inplace=True)

opids.count()

eid    141
dtype: int64

In [41]:
#Retrieve the Spark DataFrame corresponding to the fields of interest
opfieldnames = ['eid'] + operations_selfreport + operations_selfreport_dates
selfreported_op_df = participant.retrieve_fields(names=opfieldnames, engine=dxdata.connect(), coding_values='replace')

In [ ]:
selfreported_op_df.head()

Row(eid='3799870', p20004_i0_a0=None, p20004_i0_a1=None, p20004_i0_a2=None, p20004_i0_a3=None, p20004_i0_a4=None, p20004_i0_a5=None, p20004_i0_a6=None, p20004_i0_a7=None, p20004_i0_a8=None, p20004_i0_a9=None, p20004_i0_a10=None, p20004_i0_a11=None, p20004_i0_a12=None, p20004_i0_a13=None, p20004_i0_a14=None, p20004_i0_a15=None, p20004_i0_a16=None, p20004_i0_a17=None, p20004_i0_a18=None, p20004_i0_a19=None, p20004_i0_a20=None, p20004_i0_a21=None, p20004_i0_a22=None, p20004_i0_a23=None, p20004_i0_a24=None, p20004_i0_a25=None, p20004_i0_a26=None, p20004_i0_a27=None, p20004_i0_a28=None, p20004_i0_a29=None, p20004_i0_a30=None, p20004_i0_a31=None, p20004_i1_a0=None, p20004_i1_a1=None, p20004_i1_a2=None, p20004_i1_a3=None, p20004_i1_a4=None, p20004_i1_a5=None, p20004_i1_a6=None, p20004_i1_a7=None, p20004_i1_a8=None, p20004_i1_a9=None, p20004_i1_a10=None, p20004_i1_a11=None, p20004_i1_a12=None, p20004_i1_a13=None, p20004_i1_a14=None, p20004_i1_a15=None, p20004_i1_a16=None, p20004_i1_a17=None, p

In [47]:
%%capture
output_op_df = selfreported_op_df.toPandas()

/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inst

In [62]:
output_op_df.shape

(502209, 257)

In [71]:
#Filter for only individuals in opids
filtered_op_df=output_op_df[output_op_df['eid'].astype('int64').isin(opids['eid'])]

In [72]:
filtered_op_df.head()

,eid,p20004_i0_a0,p20004_i0_a1,p20004_i0_a2,p20004_i0_a3,p20004_i0_a4,p20004_i0_a5,p20004_i0_a6,p20004_i0_a7,p20004_i0_a8,...,p20010_i3_a22,p20010_i3_a23,p20010_i3_a24,p20010_i3_a25,p20010_i3_a26,p20010_i3_a27,p20010_i3_a28,p20010_i3_a29,p20010_i3_a30,p20010_i3_a31
6005,5979675,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
11501,5280005,inguinal/femoral hernia repair,pacemaker/defibrillator insertion,heart transplant,tonsillectomy +/- adenoids,wisdom teeth surgery,vasectomy,coronary angiogram,colonoscopy/sigmoidoscopy,endoscopy / gastroscopy,...,None,None,None,None,None,None,None,None,None,None
12662,3790249,urethral surgery,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13313,5283448,defibrillator/icd insertion,appendicectomy,ectopic pregnancy surgery,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
25543,1708649,defibrillator/icd insertion,tonsillectomy / tonsil surgery,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [73]:
filtered_op_df.to_csv('selfreported_opICD_HT_filtered.tsv', sep='\t', index=False)


# Extract Diagnoses (Self-Reported/ICD10/9) and Date of Diagnoses (Self-Reported/ICD10/9)

This further adds the dates of diagnoses either via using the self-reported interpolated year; the date of diagnosis from hospital diagnoses (for HCM)
OR
the date of first occurrence for HF (I50) and T2D (E11)

In [55]:
#Retrieve the Spark DataFrame corresponding to the fields of interest

non_cancer_illness_fields = sum([field_names_for_id(20002)],[]) 
non_cancer_illness_fields_interpolatedyear = sum([field_names_for_id(20008)],[]) 

icd10_dates = sum([field_names_for_id(41280)],[]) 
icd9_dates = sum([field_names_for_id(41281)],[]) 

fieldnames = ['eid', 'p41270','p41271'] + icd10_dates + icd9_dates + non_cancer_illness_fields + non_cancer_illness_fields_interpolatedyear + first_hf_occurrence + first_t2d_occurrence
selfrep_hospital_diagnoses = participant.retrieve_fields(names=fieldnames, engine=dxdata.connect(), coding_values='replace')

/tmp/ipykernel_82/339987039.py:7: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return sorted(fields, key=lambda f: LooseVersion(f.name))


In [56]:
#Read in the HF and the HCM case IDs
import pandas as pd
hf_ids = pd.read_csv('ukb_rap_HF_20240209.csv')
hcm_ids = pd.read_csv('ukb_rap_HCM_20240209.csv')
t2d_ids = pd.read_csv('ukb_rap_T2D_20240209.csv')

In [57]:
#Filter the Spark dataframe for rows corresponding to either HF cases or HCM cases (all cases from ICD10/9; self-reported; death cause record)
from pyspark.sql.functions import col

hf_selfrep_hospital_diagnoses_df = selfrep_hospital_diagnoses.filter(col('eid').isin(hf_ids.iloc[:,0].tolist()))
hcm_selfrep_hospital_diagnoses_df = selfrep_hospital_diagnoses.filter(col('eid').isin(hcm_ids.iloc[:,0].tolist()))
t2d_selfrep_hospital_diagnoses_df = selfrep_hospital_diagnoses.filter(col('eid').isin(t2d_ids.iloc[:,0].tolist()))

In [58]:
%%capture
hcm_selfrep_hospital_diagnoses_pdf = hcm_selfrep_hospital_diagnoses_df.toPandas()
hf_selfrep_hospital_diagnoses_pdf = hf_selfrep_hospital_diagnoses_df.toPandas()
t2d_selfrep_hospital_diagnoses_pdf = t2d_selfrep_hospital_diagnoses_df.toPandas()

In [60]:
hcm_selfrep_hospital_diagnoses_pdf.head()

,eid,p41270,p41271,p41280_a0,p41280_a1,p41280_a2,p41280_a3,p41280_a4,p41280_a5,p41280_a6,...,p20008_i3_a26,p20008_i3_a27,p20008_i3_a28,p20008_i3_a29,p20008_i3_a30,p20008_i3_a31,p20008_i3_a32,p20008_i3_a33,p131354,p130708
0,1026433,"[C61 Malignant neoplasm of prostate, D69.6 Thr...",None,2021-02-11,2017-12-09,2021-12-11,2017-12-09,2021-12-11,2021-12-11,2021-04-19,...,None,None,None,None,None,None,None,None,None,None
1,1032235,[E02 Subclinical iodine-deficiency hypothyroid...,None,2017-08-15,2004-10-28,2018-01-16,2017-11-28,2018-01-16,2017-11-07,2018-11-15,...,None,None,None,None,None,None,None,None,2016-05-09,None
2,1038362,"[G47.3 Sleep apnoea, I10 Essential (primary) h...",None,2013-10-16,2003-04-30,2003-04-30,2013-01-22,2013-10-16,1999-02-15,1998-12-16,...,None,None,None,None,None,None,None,None,None,None
3,1053883,[A09.9 Gastroenteritis and colitis of unspecif...,None,2014-02-04,2006-08-15,1998-06-24,2017-10-20,2017-10-20,2018-05-01,2004-07-05,...,None,None,None,None,None,None,None,None,None,2004-07-05
4,1072887,"[H61.0 Perichondritis of external ear, I20.9 A...",None,1999-02-15,2020-08-20,2021-04-03,2021-04-22,2020-08-05,2020-08-20,2021-04-09,...,None,None,None,None,None,None,None,None,None,None


In [61]:
hcm_selfrep_hospital_diagnoses_pdf.to_csv('ukb_HCM_20240209_icd10_icd9_selfrep_diagdates.tsv', sep='\t', index=False)
hf_selfrep_hospital_diagnoses_pdf.to_csv('ukb_HF_20240209_icd10_icd9_selfrep_diagdates.tsv', sep='\t', index=False)
t2d_selfrep_hospital_diagnoses_pdf.to_csv('ukb_T2D_20240209_icd10_icd9_selfrep_diagdates.tsv', sep='\t', index=False)